**Module for VIX data update**

In [9]:
from datetime import datetime, date
import pandas as pd # pandas time series library
import pandas.tseries as ts

In [19]:
m_codes = ['F','G','H','J','K','M','N','Q','U','V','X','Z'] #month codes of the futures
codes = dict(zip(range(1,13),m_codes))

def vixExpiration(year,month):
    """
    expiration date of a VX future
    
    """
    t = datetime(year,month,1)+ts.offsets.DateOffset(months=1)
    
    
    offset = ts.offsets.Week(weekday=4)
    if t.weekday()!=4:
        t_new = t+3*offset
    else:
        t_new = t+2*offset    
    
    t_exp = t_new-ts.offsets.DateOffset(days=30)
    return t_exp

def checkForUpdate(vixfut):
    '''
    check VIX index prices for latest date, return the VIX table as a dataframe if update needed
    
    '''
    existDate = vixfut.index[-1]
    expiry = vixExpiration(existDate.year,existDate.month)

    urlStr = "http://www.cboe.com/publish/ScheduledTask/MktData/datahouse/vixcurrent.csv"
    vix = pd.read_csv(urlStr,index_col=0, header=1,dayfirst=False,parse_dates=True) # read CBOE csv into dataframe
    vix = vix[vix.index > existDate]   # only keep new data rows
    
    urlStr = "http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vxvdailyprices.csv"
    vxv = pd.read_csv(urlStr,index_col=0, header=2,dayfirst=False,parse_dates=True) # read CBOE csv into dataframe
    vxv = vxv[vxv.index > existDate]   # only keep new data rows
    
    if not len(vix):
        print ("VIX futures data is up to date as of ", existDate.date())
        return
    else:
        print ("Existing: {0}  Current: {1}".format(existDate.date(),vix.index[-1].date()))
        return vix, vxv, existDate, expiry

def updateVixData(saveToPath,vixfut):
    try:
        vix_df, vxv_df, existDate, expiry = checkForUpdate(vixfut)
    except Exception as e:
        return vixfut

    vixPrices = [[]for i in range(11)]    # create empty list for each vix contract

    for j in range(len(vix_df)):   # fill in date
        vixPrices[0].append(vix_df.index[j])

    for j in range(len(vix_df)):   # fill in VIX spot price
        vixPrices[1].append(vix_df['VIX Close'][j])

    for j in range(len(vix_df)):   # fill in VXV spot price
        vixPrices[10].append(vxv_df['CLOSE'][j])
        
    # download futures prices from historical price csv's
    if len(vix_df) > 1:
        sMonth = existDate.month
        next_month = 1 if existDate > expiry else 0
        year = existDate.year
        newSettle = pd.Series(data=0)
        for i in range(8):
            month = 12 if (sMonth + i) == 12 else divmod((sMonth + i), 12)[1]
            monthSymbol = codes[month+next_month]
            next_year = 1 if (sMonth + i) > 12 else 0
            fName = "CFE_{0}{1}_VX.csv".format(monthSymbol, str(year + next_year)[-2:])
            urlStr = "http://cfe.cboe.com/Publish/ScheduledTask/MktData/datahouse/{0}".format(fName)
            df_fut = pd.read_csv(urlStr,index_col=0, header=1,dayfirst=False,parse_dates=True) # read CBOE csv into dataframe
            df_fut = df_fut[df_fut.index > existDate]
            print ('Downloaded {0}'.format(fName))
            for j in range(len(df_fut)):
                #print ('i: ', i, 'j: ',j)
                vixPrices[i+2].append(df_fut['Settle'][j])

    # download prices from the latest day
    urlStr = "http://cfe.cboe.com/data/DailyVXFuturesEODValues/DownloadFS.aspx"
    df_settle = pd.read_csv(urlStr, header=0) # read CBOE csv into dataframe

    df_settle = df_settle[df_settle.Symbol.str.startswith('VX ')]
    df_settle = df_settle.reset_index()
    settle = df_settle['SettlementPrice']

    for j in range(8):
        vixPrices[j+2].append(settle[j])

    
    d = dict(list(zip(list(vixfut), vixPrices[1:]))) # create a dictionary with vixfut-vixPrices pairs, exclude column 0
    df_new = pd.DataFrame(d, index = pd.Index(vixPrices[0]))
    df_new = df_new[list(vixfut)]   # reorder columns: move VIX to the left
    vixfut = vixfut.append(df_new)
    vixfut.to_csv(saveToPath)
    return vixfut

In [11]:
path2 = r'C:\Users\chekitsch\Documents\Trading\Historical data\VIX\term_structure.csv'
vixfut = pd.read_csv(path2, index_col=0, header = 0, parse_dates=True)

In [20]:
updateVixData(path2, vixfut)

Existing: 2017-05-11  Current: 2017-05-12


,VIX,UX1,UX2,UX3,UX4,UX5,UX6,UX7,UX8,VXV
2004-03-26,17.33,20.320,20.160,NaN,20.110,NaN,NaN,NaN,NaN,NaN
2004-03-29,16.50,19.800,19.770,NaN,19.680,NaN,NaN,NaN,NaN,NaN
2004-03-30,16.28,19.620,19.790,NaN,19.910,NaN,NaN,NaN,NaN,NaN
2004-03-31,16.74,19.750,19.850,NaN,20.150,NaN,NaN,NaN,NaN,NaN
2004-04-01,16.65,19.620,19.610,NaN,20.200,NaN,NaN,NaN,NaN,NaN
2004-04-02,15.64,18.890,19.160,NaN,20.050,NaN,NaN,NaN,NaN,NaN
2004-04-05,14.97,18.530,18.620,NaN,19.970,NaN,NaN,NaN,NaN,NaN
2004-04-06,15.32,18.790,18.790,NaN,20.280,NaN,NaN,NaN,NaN,NaN
2004-04-07,15.76,19.090,19.120,NaN,20.540,NaN,NaN,NaN,NaN,NaN
2004-04-08,16.26,19.050,19.000,NaN,20.510,NaN,NaN,NaN,NaN,NaN


In [8]:
vixExpiration(2017,5)

Timestamp('2017-05-17 00:00:00')

In [17]:
checkForUpdate(vixfut)

Existing: 2017-05-11  Current: 2017-05-12


(            VIX Open  VIX High  VIX Low  VIX Close
 Date                                              
 2017-05-12     10.72     10.87    10.28       10.4,
              OPEN   HIGH    LOW  CLOSE
 2017-05-12  13.02  13.15  12.94  12.97,
 Timestamp('2017-05-11 00:00:00'),
 Timestamp('2017-05-17 00:00:00'))